In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


file_path = "/content/drive/MyDrive/Colab_Notebooks/Mapped_Dataset.csv"

global train_gen
def data_generator(file_path, batch_size=512, num_classes=10):
    while True:  # Vòng lặp vô hạn
        chunks = pd.read_csv(file_path, chunksize=batch_size)
        for chunk in chunks:
            X = chunk.iloc[:, :-1].values.reshape(-1, 46, 1)  # Định dạng input
            y = chunk.iloc[:, -1].values
            y_onehot = to_categorical(y, num_classes=num_classes)
            yield X, y_onehot

    # chunks = pd.read_csv(file_path, chunksize=batch_size)

    # for chunk in chunks:
    #     X = chunk.iloc[:, :-1].values.reshape(-1, 46, 1)  # Định dạng input
    #     y = chunk.iloc[:, -1].values
    #     y_onehot = to_categorical(y, num_classes=num_classes)
    #     yield X, y_onehot




In [ ]:
train_gen = data_generator(file_path, batch_size=64, num_classes=10)

from tensorflow import keras
# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường

model = keras.Sequential([
    layers.Input(shape=(46,1)),
    layers.Conv1D(filters=32, kernel_size=3, padding="same",activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0,5),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_gen, epochs=10, steps_per_epoch=720000)

# Lưu mô hình
model.save("cnn_model.keras")

Epoch 1/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 823s 9ms/step - accuracy: 0.7819 - loss: 0.6012
Epoch 2/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 804s 9ms/step - accuracy: 0.8108 - loss: 0.4828
Epoch 3/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 761s 8ms/step - accuracy: 0.8224 - loss: 0.4491
Epoch 4/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 758s 8ms/step - accuracy: 0.8484 - loss: 0.3764
Epoch 5/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 739s 8ms/step - accuracy: 0.8633 - loss: 0.3473
Epoch 6/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 758s 8ms/step - accuracy: 0.8687 - loss: 0.3329
Epoch 7/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 701s 8ms/step - accuracy: 0.8720 - loss: 0.3200
Epoch 8/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 743s 8ms/step - accuracy: 0.8732 - loss: 0.3115
Epoch 9/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 721s 8ms/step - accuracy: 0.8751 - loss: 0.3040
Epoch 10/10
90000/90000 ━━━━━━━━━━━━━━━━━━━━ 752s 8ms/step - accuracy: 0.8760 - loss: 0.3002
